In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive

drive.mount("/content/drive")

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = "cs231n/assignments/assignment2/"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys

sys.path.append("/content/drive/My Drive/{}".format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/cs231n/assignments/assignment2/cs231n/datasets
/content/drive/My Drive/cs231n/assignments/assignment2


ローカルで実行する場合

In [1]:
%cd ./cs231n/datasets/
!bash get_datasets.sh
%cd ../../

/workspaces/cs231n/src/assignment2/cs231n/datasets
/workspaces/cs231n/src/assignment2


# Introduction to PyTorch

You've written a lot of code in this assignment to provide a whole host of neural network functionality. Dropout, Batch Norm, and 2D convolutions are some of the workhorses of deep learning in computer vision. You've also worked hard to make your code efficient and vectorized.

For the last part of this assignment, though, we're going to leave behind your beautiful codebase and instead migrate to one of two popular deep learning frameworks: in this instance, PyTorch (or TensorFlow, if you choose to work with that notebook).

この課題では、ニューラル ネットワークのさまざまな機能を提供するために、大量のコードを作成しました。ドロップアウト、バッチ正規化、2D 畳み込みは、コンピューター ビジョンにおけるディープラーニングの主力機能の一部です。また、コードを効率化してベクトル化するためにも努力しました。

ただし、この課題の最後の部分では、美しいコードベースを離れ、代わりに 2 つの一般的なディープラーニング フレームワークのいずれかに移行します。この例では、PyTorch (または、ノートブックで作業することを選択した場合は TensorFlow) です。

## Why do we use deep learning frameworks?

* Our code will now run on GPUs! This will allow our models to train much faster. When using a framework like PyTorch or TensorFlow you can harness the power of the GPU for your own custom neural network architectures without having to write CUDA code directly (which is beyond the scope of this class).
* In this class, we want you to be ready to use one of these frameworks for your project so you can experiment more efficiently than if you were writing every feature you want to use by hand. 
* We want you to stand on the shoulders of giants! TensorFlow and PyTorch are both excellent frameworks that will make your lives a lot easier, and now that you understand their guts, you are free to use them :) 
* Finally, we want you to be exposed to the sort of deep learning code you might run into in academia or industry.

* コードは GPU 上で実行されるようになりました。これにより、モデルのトレーニングがはるかに高速になります。PyTorch や TensorFlow などのフレームワークを使用すると、CUDA コードを直接記述することなく (これはこのクラスの範囲外です)、GPU のパワーを独自のカスタム ニューラル ネットワーク アーキテクチャに活用できます。
* このクラスでは、これらのフレームワークのいずれかをプロジェクトで使用できるようにして、使用したいすべての機能を手動で記述する場合よりも効率的に実験できるようにします。
* 巨人の肩の上に立ってほしいです。TensorFlow と PyTorch はどちらも優れたフレームワークで、あなたの生活を大幅に楽にしてくれます。その仕組みを理解した今、あなたは自由にそれらを使用できます :)
* 最後に、学術界や業界で遭遇する可能性のある種類のディープラーニング コードに触れてほしいです。

## What is PyTorch?

PyTorch is a system for executing dynamic computational graphs over Tensor objects that behave similarly as numpy ndarray. It comes with a powerful automatic differentiation engine that removes the need for manual back-propagation. 

PyTorch は、numpy ndarray と同様に動作する Tensor オブジェクト上で動的な計算グラフを実行するシステムです。強力な自動微分エンジンが搭載されており、手動によるバックプロパゲーションが不要になります。

## How do I learn PyTorch?

One of our former instructors, Justin Johnson, made an excellent [tutorial](https://github.com/jcjohnson/pytorch-examples) for PyTorch. 

You can also find the detailed [API doc](http://pytorch.org/docs/stable/index.html) here. If you have other questions that are not addressed by the API docs, the [PyTorch forum](https://discuss.pytorch.org/) is a much better place to ask than StackOverflow.

元インストラクターの 1 人である Justin Johnson が、PyTorch の優れた [チュートリアル](https://github.com/jcjohnson/pytorch-examples) を作成しました。

詳細な [API ドキュメント](http://pytorch.org/docs/stable/index.html) もこちらでご覧いただけます。API ドキュメントで取り上げられていないその他の質問がある場合は、StackOverflow よりも [PyTorch フォーラム](https://discuss.pytorch.org/) の方がはるかに適切な質問の場です。

# Table of Contents

This assignment has 5 parts. You will learn PyTorch on **three different levels of abstraction**, which will help you understand it better and prepare you for the final project. 

1. Part I, Preparation: we will use CIFAR-10 dataset.
2. Part II, Barebones PyTorch: **Abstraction level 1**, we will work directly with the lowest-level PyTorch Tensors. 
3. Part III, PyTorch Module API: **Abstraction level 2**, we will use `nn.Module` to define arbitrary neural network architecture. 
4. Part IV, PyTorch Sequential API: **Abstraction level 3**, we will use `nn.Sequential` to define a linear feed-forward network very conveniently. 
5. Part V, CIFAR-10 open-ended challenge: please implement your own network to get as high accuracy as possible on CIFAR-10. You can experiment with any layer, optimizer, hyperparameters or other advanced features. 

Here is a table of comparison:

| API           | Flexibility | Convenience |
|---------------|-------------|-------------|
| Barebone      | High        | Low         |
| `nn.Module`     | High        | Medium      |
| `nn.Sequential` | Low         | High        |

この課題は 5 つのパートに分かれています。**3 つの異なる抽象化レベル** で PyTorch を学習することで、PyTorch をより深く理解し、最終プロジェクトの準備ができます。

1. パート I、準備: CIFAR-10 データセットを使用します。

2. パート II、ベアボーン PyTorch: **抽象化レベル 1**、最も低いレベルの PyTorch テンソルを直接操作します。

3. パート III、PyTorch モジュール API: **抽象化レベル 2**、`nn.Module` を使用して任意のニューラル ネットワーク アーキテクチャを定義します。

4. パート IV、PyTorch Sequential API: **抽象化レベル 3**、`nn.Sequential` を使用して非常に便利な線形フィードフォワード ネットワークを定義します。
5. パート V、CIFAR-10 オープンエンド チャレンジ: CIFAR-10 で可能な限り高い精度を得るために、独自のネットワークを実装してください。任意のレイヤー、オプティマイザー、ハイパーパラメータ、その他の高度な機能を試すことができます。

比較表を以下に示します。

| API | 柔軟性 | 利便性 |
|--------------|-------------|--------------|
| ベアボーン | 高 | 低 |
| `nn.Module` | 高 | 中 |
| `nn.Sequential` | 低 | 高 |

# GPU

You can manually switch to a GPU device on Colab by clicking `Runtime -> Change runtime type` and selecting `GPU` under `Hardware Accelerator`. You should do this before running the following cells to import packages, since the kernel gets restarted upon switching runtimes.

Colab で GPU デバイスに手動で切り替えるには、「ランタイム -> ランタイム タイプを変更」をクリックし、「ハードウェア アクセラレータ」の下の「GPU」を選択します。ランタイムを切り替えるとカーネルが再起動されるため、次のセルを実行してパッケージをインポートする前にこれを行う必要があります。

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

USE_GPU = True
dtype = torch.float32  # このチュートリアルでは float を使用します。

if USE_GPU and torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# トレーニング損失をprintする頻度を制御する定数。
print_every = 100
print("using device:", device)

using device: cuda


# Part I. Preparation

Now, let's load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In previous parts of the assignment we had to write our own code to download the CIFAR-10 dataset, preprocess it, and iterate through it in minibatches; PyTorch provides convenient tools to automate this process for us.

それでは、CIFAR-10 データセットをロードしましょう。初めて行う場合は数分かかるかもしれませんが、その後はファイルはキャッシュされたままになります。

課題の前のパートでは、CIFAR-10 データセットをダウンロードし、前処理して、ミニバッチで反復処理するための独自のコードを記述する必要がありましたが、PyTorch には、このプロセスを自動化するための便利なツールが用意されています。

In [3]:
NUM_TRAIN = 49000

# torchvision.transforms パッケージは、データの前処理とデータ拡張を実行するためのツールを提供します。
# ここでは、平均 RGB 値を減算し、各 RGB 値の標準偏差で割ることでデータを前処理する変換を設定します。
# 平均と標準偏差はハードコードされています。
transform = T.Compose(
    [T.ToTensor(), T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
)

# データセットは一度に1つずつトレーニング例をロードするので、
# データセットを反復処理してミニバッチを形成するDataLoaderで各データセットをラップします。
# SamplerオブジェクトをDataLoaderに渡すことで、CIFAR-10の学習セットをtrainセットとvalセットに分割します。
cifar10_train = dset.CIFAR10(
    "./cs231n/datasets", train=True, download=True, transform=transform
)
loader_train = DataLoader(
    cifar10_train, batch_size=64, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN))
)

cifar10_val = dset.CIFAR10(
    "./cs231n/datasets", train=True, download=True, transform=transform
)
loader_val = DataLoader(
    cifar10_val,
    batch_size=64,
    sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)),
)

cifar10_test = dset.CIFAR10(
    "./cs231n/datasets", train=False, download=True, transform=transform
)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Part II. Barebones PyTorch

PyTorch ships with high-level APIs to help us define model architectures conveniently, which we will cover in Part II of this tutorial. In this section, we will start with the barebone PyTorch elements to understand the autograd engine better. After this exercise, you will come to appreciate the high-level model API more.

We will start with a simple fully-connected ReLU network with two hidden layers and no biases for CIFAR classification. 
This implementation computes the forward pass using operations on PyTorch Tensors, and uses PyTorch autograd to compute gradients. It is important that you understand every line, because you will write a harder version after the example.

When we create a PyTorch Tensor with `requires_grad=True`, then operations involving that Tensor will not just compute values; they will also build up a computational graph in the background, allowing us to easily backpropagate through the graph to compute gradients of some Tensors with respect to a downstream loss. Concretely if x is a Tensor with `x.requires_grad == True` then after backpropagation `x.grad` will be another Tensor holding the gradient of x with respect to the scalar loss at the end.

PyTorchにはモデルアーキテクチャを簡単に定義するための高レベルAPIが同梱されています。このセクションでは、autogradエンジンをよりよく理解するために、PyTorchの素の要素から始めます。この練習の後、高レベルのモデルAPIをより理解できるようになるでしょう。

CIFAR分類のために、2つの隠れ層とバイアスを持たない単純な完全連結ReLUネットワークから始めます。
この実装では、PyTorch Tensorsの操作を使ってフォワードパスを計算し、PyTorch autogradを使って勾配を計算します。すべての行を理解することが重要です。

`requires_grad=True`でPyTorch Tensorを作成すると、そのTensorを含む操作は単に値を計算するだけでなく、バックグラウンドで計算グラフを構築します。具体的には、xが`x.requires_grad == True`のテンソルである場合、バックプロパゲーションの後、`x.grad`はスカラー損失に対するxの勾配を保持する別のテンソルになります。

### PyTorch Tensors: Flatten Function
A PyTorch Tensor is conceptionally similar to a numpy array: it is an n-dimensional grid of numbers, and like numpy PyTorch provides many functions to efficiently operate on Tensors. As a simple example, we provide a `flatten` function below which reshapes image data for use in a fully-connected neural network.

Recall that image data is typically stored in a Tensor of shape N x C x H x W, where:

* N is the number of datapoints
* C is the number of channels
* H is the height of the intermediate feature map in pixels
* W is the height of the intermediate feature map in pixels

This is the right way to represent the data when we are doing something like a 2D convolution, that needs spatial understanding of where the intermediate features are relative to each other. When we use fully connected affine layers to process the image, however, we want each datapoint to be represented by a single vector -- it's no longer useful to segregate the different channels, rows, and columns of the data. So, we use a "flatten" operation to collapse the `C x H x W` values per representation into a single long vector. The flatten function below first reads in the N, C, H, and W values from a given batch of data, and then returns a "view" of that data. "View" is analogous to numpy's "reshape" method: it reshapes x's dimensions to be N x ??, where ?? is allowed to be anything (in this case, it will be C x H x W, but we don't need to specify that explicitly). 

PyTorch Tensorは概念的にはnumpyの配列に似ています。numpyのようにPyTorchはTensorを効率的に操作するための多くの関数を提供します。簡単な例として、完全連結ニューラルネットワークで使用するために画像データを整形する `flatten` 関数を以下に示します。

画像データは通常N x C x H x Wのテンソルに格納されていることを思い出してください：

* N はデータポイントの数である。
* C はチャネルの数
* Hは中間特徴マップの高さ（ピクセル）である。
* W は中間特徴マップの高さ（ピクセル）である。

これは、2次元畳み込みのような、中間特徴同士の相対的な位置関係を空間的に理解する必要がある場合に、データを表現する正しい方法である。しかし、完全連結アフィンレイヤーを使って画像を処理する場合は、各データポイントを1つのベクトルで表現したい。そこで、「flatten 」操作を使って、表現ごとに`C x H x W`の値を1つの長いベクトルにまとめる。以下のflatten関数は、まず与えられたデータのバッチからN、C、H、Wの値を読み込み、そのデータの 「ビュー 」を返します。「view 」はnumpyの 「reshape 」メソッドに類似しています。「view 」はxの次元をN×?

In [4]:
def flatten(x):
    # N、C、H、Wで読む
    N = x.shape[0]
    # C * H * Wの値を画像ごとに1つのベクトルに 「平坦化 」する。
    return x.view(N, -1)


def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print("Before flattening: ", x)
    print("After flattening: ", flatten(x))


test_flatten()

Before flattening:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])
After flattening:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


### Barebones PyTorch: Two-Layer Network

Here we define a function `two_layer_fc` which performs the forward pass of a two-layer fully-connected ReLU network on a batch of image data. After defining the forward pass we check that it doesn't crash and that it produces outputs of the right shape by running zeros through the network.

You don't have to write any code here, but it's important that you read and understand the implementation.

ここでは、画像データのバッチに対して2層完全連結ReLUネットワークのフォワードパスを実行する関数 `two_layer_fc` を定義する。フォワードパスを定義した後、それがクラッシュしないことと、ネットワークにゼロを通すことで正しい形状の出力を生成することをチェックする。

ここでコードを書く必要はありませんが、実装を読んで理解することが重要です。

In [5]:
import torch.nn.functional as F  # 便利なステートレス関数


def two_layer_fc(x, params):
    """
    A fully-connected neural networks; the architecture is:
    NN is fully connected -> ReLU -> fully connected layer.
    Note that this function only defines the forward pass;
    PyTorch will take care of the backward pass for us.

    The input to the network will be a minibatch of data, of shape
    (N, d1, ..., dM) where d1 * ... * dM = D. The hidden layer will have H units,
    and the output layer will produce scores for C classes.

    Inputs:
    - x: A PyTorch Tensor of shape (N, d1, ..., dM) giving a minibatch of
      input data.
    - params: A list [w1, w2] of PyTorch Tensors giving weights for the network;
      w1 has shape (D, H) and w2 has shape (H, C).

    Returns:
    - scores: A PyTorch Tensor of shape (N, C) giving classification scores for
      the input data x.

    完全連結ニューラルネットワーク：
    NNは完全連結→ReLU→完全連結層。
    この関数はフォワードパスだけを定義していることに注意してください；
    後方パスはPyTorchがやってくれます。

    ネットワークへの入力は、次のような形のデータのミニバッチになります。
    (N, d1, ..., dM)で、d1 * ... * dM = Dです。* 隠れ層にはH個のユニットがある、
    出力層はCクラスのスコアを生成する。

    入力
    - x: 入力： x: 形状 (N, d1, ..., dM)のPyTorchテンソル。
      入力データ。
    - params: params: ネットワークの重みを与えるPyTorchテンソルのリスト[w1, w2];
      w1 の形状は (D, H)、w2 の形状は (H, C) です。

    戻り値
    - スコア： 入力データ x の分類スコアを与える shape (N, C) の PyTorch Tensor
    """
    # まず画像を平らにする
    # shape: [batch_size, C x H x W]
    x = flatten(x)

    w1, w2 = params

    # フォワード・パス：テンソルの演算を使用して予測yを計算する。
    # w1とw2はrequires_grad=Trueなので、これらのテンソルを含む操作は
    # PyTorchに計算グラフを構築させ、勾配の自動計算を可能にします。
    # バックワードパスを手で実装する必要がなくなったので、中間値の参照を保持する必要がなくなりました。
    x = F.relu(x.mm(w1))
    x = x.mm(w2)
    return x


def two_layer_fc_test():
    hidden_layer_size = 42
    x = torch.zeros((64, 50), dtype=dtype)  # ミニバッチサイズ64、特徴次元50
    w1 = torch.zeros((50, hidden_layer_size), dtype=dtype)
    w2 = torch.zeros((hidden_layer_size, 10), dtype=dtype)
    scores = two_layer_fc(x, [w1, w2])
    print(scores.size())  # [64,10]を見てほしい。


two_layer_fc_test()

torch.Size([64, 10])


### Barebones PyTorch: Three-Layer ConvNet

Here you will complete the implementation of the function `three_layer_convnet`, which will perform the forward pass of a three-layer convolutional network. Like above, we can immediately test our implementation by passing zeros through the network. The network should have the following architecture:

1. A convolutional layer (with bias) with `channel_1` filters, each with shape `KW1 x KH1`, and zero-padding of two
2. ReLU nonlinearity
3. A convolutional layer (with bias) with `channel_2` filters, each with shape `KW2 x KH2`, and zero-padding of one
4. ReLU nonlinearity
5. Fully-connected layer with bias, producing scores for C classes.

Note that we have **no softmax activation** here after our fully-connected layer: this is because PyTorch's cross entropy loss performs a softmax activation for you, and by bundling that step in makes computation more efficient.

**HINT**: For convolutions: http://pytorch.org/docs/stable/nn.html#torch.nn.functional.conv2d; pay attention to the shapes of convolutional filters!

ここでは、3層畳み込みネットワークのフォワードパスを実行する関数 `three_layer_convnet` の実装を完了する。上と同じように、ネットワークに0を渡すことで実装をすぐにテストできる。このネットワークは次のようなアーキテクチャーを持つ：

1. チャンネル_1`のフィルターを持つ畳み込み層（バイアス付き）。
2. ReLU非線形性
3. チャンネル_2`の畳み込み層（バイアス付き）。
4. ReLU非線形性
5. Cクラスのスコアを生成する、バイアスを持つ全結合層。

これは、PyTorchのクロスエントロピー損失がソフトマックス活性化を行うためで、このステップをバンドルすることで計算がより効率的になります。

**HINT**： 畳み込みの場合: http://pytorch.org/docs/stable/nn.html#torch.nn.functional.conv2d; 畳み込みフィルタの形に注意！

In [6]:
def three_layer_convnet(x, params):
    """
    Performs the forward pass of a three-layer convolutional network with the
    architecture defined above.

    Inputs:
    - x: A PyTorch Tensor of shape (N, 3, H, W) giving a minibatch of images
    - params: A list of PyTorch Tensors giving the weights and biases for the
      network; should contain the following:
      - conv_w1: PyTorch Tensor of shape (channel_1, 3, KH1, KW1) giving weights
        for the first convolutional layer
      - conv_b1: PyTorch Tensor of shape (channel_1,) giving biases for the first
        convolutional layer
      - conv_w2: PyTorch Tensor of shape (channel_2, channel_1, KH2, KW2) giving
        weights for the second convolutional layer
      - conv_b2: PyTorch Tensor of shape (channel_2,) giving biases for the second
        convolutional layer
      - fc_w: PyTorch Tensor giving weights for the fully-connected layer. Can you
        figure out what the shape should be?
      - fc_b: PyTorch Tensor giving biases for the fully-connected layer. Can you
        figure out what the shape should be?

    Returns:
    - scores: PyTorch Tensor of shape (N, C) giving classification scores for x

    上で定義したアーキテクチャを持つ3層畳み込みネットワークのフォワードパスを実行する。
    フォワードパスを実行する。

    入力
    - x: 画像のミニバッチを与える形状 (N, 3, H, W) の PyTorchテンソル
    - params: ネットワークの重みとバイアスを与えるPyTorchテンソルのリスト。
      以下を含む必要があります：
      - conv_w1: conv_w1: 形状 (channel_1, 3, KH1, KW1) のPyTorchテンソル。
        を与える。
      - conv_b1: 形状 (channel_1,) のPyTorchテンソル。
        畳み込み層
      - conv_w2: 形状のPyTorchテンソル (channel_2, channel_1, KH2, KW2)。
        2番目の畳み込み層の重みを与える
      - conv_b2: 2番目の畳み込み層のバイアスを与える形状 (channel_2,) のPyTorchテンソル
        畳み込み層
      - fc_w: 完全連結層の重みを与えるPyTorchテンソル。あなたは
        どのような形状かわかるだろうか？
      - fc_b: 完全連結層のバイアスを与えるPyTorchテンソル。どのような
        どのような形状になるかわかりますか？

    を返します：
    - scores: x の分類スコアを与える (N, C) 形式の PyTorch テンソル
    """
    conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b = params
    scores = None
    ################################################################################
    # TODO: Implement the forward pass for the three-layer ConvNet.                #
    #                                                                              #
    # 3層ConvNetのフォワードパスを実装する。
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    x = F.relu(F.conv2d(x, conv_w1, conv_b1, padding=2))
    x = F.relu(F.conv2d(x, conv_w2, conv_b2, padding=1))
    scores = flatten(x).mm(fc_w) + fc_b

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ################################################################################
    #                                 END OF YOUR CODE                             #
    ################################################################################
    return scores

After defining the forward pass of the ConvNet above, run the following cell to test your implementation.

When you run this function, scores should have shape (64, 10).

上記の ConvNet のフォワードパスを定義した後、以下のセルを実行して実装をテストする。

この関数を実行すると、スコアが (64, 10) の形になるはずである。

In [7]:
def three_layer_convnet_test():
    x = torch.zeros(
        (64, 3, 32, 32), dtype=dtype
    )  # ミニバッチサイズ64、画像サイズ[3, 32, 32］

    conv_w1 = torch.zeros(
        (6, 3, 5, 5), dtype=dtype
    )  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b1 = torch.zeros((6,))  # out_channel
    conv_w2 = torch.zeros(
        (9, 6, 3, 3), dtype=dtype
    )  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b2 = torch.zeros((9,))  # out_channel

    # 2つの畳み込み層の後、完全連結層の前に、テンソルの形状を計算しなければならない。
    fc_w = torch.zeros((9 * 32 * 32, 10))
    fc_b = torch.zeros(10)

    scores = three_layer_convnet(x, [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b])
    print(scores.size())  # [64,10]を見てほしい。


three_layer_convnet_test()

torch.Size([64, 10])


### Barebones PyTorch: Initialization
Let's write a couple utility methods to initialize the weight matrices for our models.

- `random_weight(shape)` initializes a weight tensor with the Kaiming normalization method.
- `zero_weight(shape)` initializes a weight tensor with all zeros. Useful for instantiating bias parameters.

The `random_weight` function uses the Kaiming normal initialization method, described in:

モデルの重み行列を初期化するユーティリティ・メソッドをいくつか書いてみよう。

- random_weight(shape)` は重みテンソルを Kaiming 正規化法で初期化します。
- zero_weight(shape)` は重みテンソルをゼロで初期化します。バイアスパラメータを初期化するのに便利である。

random_weight` 関数は Kaiming 正規化法を用いて重みテンソルを初期化する：

He et al, *Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification*, ICCV 2015, https://arxiv.org/abs/1502.01852

In [8]:
def random_weight(shape):
    """
    Create random Tensors for weights; setting requires_grad=True means that we
    want to compute gradients for these Tensors during the backward pass.
    We use Kaiming normalization: sqrt(2 / fan_in)

    requires_grad=Trueに設定することは、後方パス中にこれらのテンソルの勾配を計算することを意味します。
    requires_grad=Trueは、バックワードパス中にこれらのテンソルの勾配を計算することを意味します。
    カイミング正規化を使用する： sqrt(2 / fan_in)
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[0]
    else:
        fan_in = np.prod(shape[1:])  # conv weight [out_channel, in_channel, kH, kW]
    # randnは標準正規分布生成器。
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2.0 / fan_in)
    w.requires_grad = True
    return w


def zero_weight(shape):
    return torch.zeros(shape, device=device, dtype=dtype, requires_grad=True)


# [3 x 5]の形状のウェイトを作成する。
# GPUを使用している場合は、`torch.cuda.FloatTensor`型が表示されるはずです。
# そうでない場合は `torch.FloatTensor` となる。
random_weight((3, 5))

tensor([[-0.0746, -0.5672,  0.6610, -0.1999,  0.5565],
        [ 0.5527,  0.4271, -0.0703, -0.4269,  1.5827],
        [-0.2340, -0.9828,  1.5024, -0.0994, -1.0913]], device='cuda:0',
       requires_grad=True)

### Barebones PyTorch: Check Accuracy
When training the model we will use the following function to check the accuracy of our model on the training or validation sets.

When checking accuracy we don't need to compute any gradients; as a result we don't need PyTorch to build a computational graph for us when we compute scores. To prevent a graph from being built we scope our computation under a `torch.no_grad()` context manager.

モデルを学習するとき、学習セットまたは検証セットでモデルの精度をチェックするために以下の関数を使います。

精度をチェックするとき、勾配を計算する必要はありません。その結果、スコアを計算するときにPyTorchが計算グラフを構築する必要はありません。グラフが作られないようにするために、 `torch.no_grad()` コンテキストマネージャの下で計算をスコープします。

In [10]:
def check_accuracy_part2(loader, model_fn, params):
    """
    Check the accuracy of a classification model.

    Inputs:
    - loader: A DataLoader for the data split we want to check
    - model_fn: A function that performs the forward pass of the model,
      with the signature scores = model_fn(x, params)
    - params: List of PyTorch Tensors giving parameters of the model

    Returns: Nothing, but prints the accuracy of the model

    分類モデルの精度をチェックする。

    入力
    - loader: チェックしたいデータ分割のDataLoader。
    - model_fn: モデルのフォワードパスを実行する関数、
      score = model_fn(x, params) のシグネチャを持つ。
    - params: モデルのパラメータを与えるPyTorchテンソルのリスト

    戻り値 何も返さないが、モデルの精度を表示する
    """
    split = "val" if loader.dataset.train else "test"
    print("Checking accuracy on the %s set" % split)
    num_correct, num_samples = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # GPUなどのデバイスに移す
            y = y.to(device=device, dtype=torch.int64)
            scores = model_fn(x, params)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print("Got %d / %d correct (%.2f%%)" % (num_correct, num_samples, 100 * acc))

### BareBones PyTorch: Training Loop
We can now set up a basic training loop to train our network. We will train the model using stochastic gradient descent without momentum. We will use `torch.functional.cross_entropy` to compute the loss; you can [read about it here](http://pytorch.org/docs/stable/nn.html#cross-entropy).

The training loop takes as input the neural network function, a list of initialized parameters (`[w1, w2]` in our example), and learning rate.

これで、ネットワークを訓練するための基本的な訓練ループを設定できる。モメンタムを用いない確率的勾配降下法を用いてモデルを学習する。損失を計算するために `torch.functional.cross_entropy` を使用します。[詳しくはこちら](http://pytorch.org/docs/stable/nn.html#cross-entropy) を参照してください。

学習ループはニューラルネットワーク関数、初期化されたパラメータのリスト（この例では `[w1, w2]`）、学習率を入力として受け取ります。

In [11]:
def train_part2(model_fn, params, learning_rate):
    """
    Train a model on CIFAR-10.

    Inputs:
    - model_fn: A Python function that performs the forward pass of the model.
      It should have the signature scores = model_fn(x, params) where x is a
      PyTorch Tensor of image data, params is a list of PyTorch Tensors giving
      model weights, and scores is a PyTorch Tensor of shape (N, C) giving
      scores for the elements in x.
    - params: List of PyTorch Tensors giving weights for the model
    - learning_rate: Python scalar giving the learning rate to use for SGD

    Returns: Nothing

    CIFAR-10でモデルをトレーニングする。

    入力
    - model_fn: モデルのフォワードパスを行う Python 関数。
      score = model_fn(x, params) のシグネチャを持つ。
      params はモデルの重みを与える PyTorch Tensor のリストで
      モデルの重みを与えるPyTorchテンソルのリストであり、scoresはxの要素のスコアを与える形状(N, C)のPyTorchテンソルです。
      のPyTorchテンソルである。
    - params: モデルの重みを与えるPyTorchテンソルのリスト
    - learning_rate: SGDに使用する学習率を与えるPythonスカラー

    戻り値 何も返さない
    """
    for t, (x, y) in enumerate(loader_train):
        # データを適切なデバイス（GPUまたはCPU）に移す
        x = x.to(device=device, dtype=dtype)
        y = y.to(device=device, dtype=torch.long)

        # フォワードパス：得点と失点を計算
        scores = model_fn(x, params)
        loss = F.cross_entropy(scores, y)

        # バックワードパス：
        # PyTorchは、計算グラフ内のどのテンソルがrequires_grad=Trueであるかを調べ、
        # バックプロパゲーションを使用して、これらのテンソルに関する損失の勾配を計算し、
        # 各テンソルの.grad属性に勾配を格納します。
        loss.backward()

        # パラメータの更新。パラメータの更新をバックプロパゲートしたくないので、
        # torch.no_grad()コンテキスト・マネージャーの下で更新をスコープし、
        # 計算グラフが構築されるのを防いでいる。
        with torch.no_grad():
            for w in params:
                w -= learning_rate * w.grad

                # バックワードパスを実行した後、手動で勾配をゼロにする。
                w.grad.zero_()

        if t % print_every == 0:
            print("Iteration %d, loss = %.4f" % (t, loss.item()))
            check_accuracy_part2(loader_val, model_fn, params)
            print()

### BareBones PyTorch: Train a Two-Layer Network
Now we are ready to run the training loop. We need to explicitly allocate tensors for the fully connected weights, `w1` and `w2`. 

Each minibatch of CIFAR has 64 examples, so the tensor shape is `[64, 3, 32, 32]`. 

After flattening, `x` shape should be `[64, 3 * 32 * 32]`. This will be the size of the first dimension of `w1`. 
The second dimension of `w1` is the hidden layer size, which will also be the first dimension of `w2`. 

Finally, the output of the network is a 10-dimensional vector that represents the probability distribution over 10 classes. 

You don't need to tune any hyperparameters but you should see accuracies above 40% after training for one epoch.

これで学習ループを実行する準備ができた。全結合の重みである `w1` と `w2` のテンソルを明示的に確保する必要がある。

CIFARの各ミニバッチは64個の例を持つので、テンソルの形状は `[64, 3, 32, 32]` となる。

平坦化後の `x` の形状は `[64, 3 * 32 * 32]` となる。これが `w1` の最初の次元のサイズとなる。
w1`の2番目の次元は隠れ層のサイズで、これも`w2`の1番目の次元となる。

最後に、ネットワークの出力は10クラスの確率分布を表す10次元のベクトルです。

ハイパーパラメータを調整する必要はありませんが、1エポックの学習で40%以上の精度が得られるはずです。

In [12]:
hidden_layer_size = 4000
learning_rate = 1e-2

w1 = random_weight((3 * 32 * 32, hidden_layer_size))
w2 = random_weight((hidden_layer_size, 10))

train_part2(two_layer_fc, [w1, w2], learning_rate)

Iteration 0, loss = 4.3062
Checking accuracy on the val set
Got 131 / 1000 correct (13.10%)

Iteration 100, loss = 2.0468
Checking accuracy on the val set
Got 332 / 1000 correct (33.20%)

Iteration 200, loss = 1.9145
Checking accuracy on the val set
Got 379 / 1000 correct (37.90%)

Iteration 300, loss = 2.2124
Checking accuracy on the val set
Got 380 / 1000 correct (38.00%)

Iteration 400, loss = 2.1293
Checking accuracy on the val set
Got 386 / 1000 correct (38.60%)

Iteration 500, loss = 2.2714
Checking accuracy on the val set
Got 415 / 1000 correct (41.50%)

Iteration 600, loss = 1.7576
Checking accuracy on the val set
Got 458 / 1000 correct (45.80%)

Iteration 700, loss = 1.6283
Checking accuracy on the val set
Got 415 / 1000 correct (41.50%)



### BareBones PyTorch: Training a ConvNet

In the below you should use the functions defined above to train a three-layer convolutional network on CIFAR. The network should have the following architecture:

1. Convolutional layer (with bias) with 32 5x5 filters, with zero-padding of 2
2. ReLU
3. Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
4. ReLU
5. Fully-connected layer (with bias) to compute scores for 10 classes

You should initialize your weight matrices using the `random_weight` function defined above, and you should initialize your bias vectors using the `zero_weight` function above.

You don't need to tune any hyperparameters, but if everything works correctly you should achieve an accuracy above 42% after one epoch.

以下では、上記で定義した関数を使用して、CIFAR 上で 3 層畳み込みネットワークを学習します。このネットワークは以下のようなアーキテクチャーでなければならない：

1. 畳み込み層（バイアスあり）、5x5 フィルタ 32 個、ゼロパディング 2 個。
2. ReLU
3. 16個の3x3フィルターによる畳み込み層（バイアスあり）。
4. ReLU
5. 完全連結層（バイアスあり）10クラスのスコアを計算する。

重み行列は上で定義した `random_weight` 関数を使って初期化し、バイアスベクトルは上で定義した `zero_weight` 関数を使って初期化します。

ハイパーパラメータを調整する必要はありませんが、すべてが正しく動作すれば、1エポック後に42%以上の精度を達成できるはずです。

In [13]:
learning_rate = 3e-3

channel_1 = 32
channel_2 = 16

conv_w1 = None
conv_b1 = None
conv_w2 = None
conv_b2 = None
fc_w = None
fc_b = None

################################################################################
# TODO: Initialize the parameters of a three-layer ConvNet.                    #
#                                                                              #
# 3層ConvNetのパラメータを初期化する。
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

conv_w1 = random_weight((channel_1, 3, 5, 5))
conv_b1 = zero_weight(channel_1)
conv_w2 = random_weight((channel_2, channel_1, 3, 3))
conv_b2 = zero_weight(channel_2)
fc_w = random_weight((channel_2 * 32 * 32, 10))
fc_b = zero_weight(10)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

params = [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b]
train_part2(three_layer_convnet, params, learning_rate)

Iteration 0, loss = 2.6533
Checking accuracy on the val set
Got 94 / 1000 correct (9.40%)

Iteration 100, loss = 1.8670
Checking accuracy on the val set
Got 332 / 1000 correct (33.20%)

Iteration 200, loss = 1.9479
Checking accuracy on the val set
Got 400 / 1000 correct (40.00%)

Iteration 300, loss = 1.5512
Checking accuracy on the val set
Got 416 / 1000 correct (41.60%)

Iteration 400, loss = 1.5163
Checking accuracy on the val set
Got 438 / 1000 correct (43.80%)

Iteration 500, loss = 1.5558
Checking accuracy on the val set
Got 448 / 1000 correct (44.80%)

Iteration 600, loss = 1.5796
Checking accuracy on the val set
Got 455 / 1000 correct (45.50%)

Iteration 700, loss = 1.5087
Checking accuracy on the val set
Got 474 / 1000 correct (47.40%)



# Part III. PyTorch Module API

Barebone PyTorch requires that we track all the parameter tensors by hand. This is fine for small networks with a few tensors, but it would be extremely inconvenient and error-prone to track tens or hundreds of tensors in larger networks.

PyTorch provides the `nn.Module` API for you to define arbitrary network architectures, while tracking every learnable parameters for you. In Part II, we implemented SGD ourselves. PyTorch also provides the `torch.optim` package that implements all the common optimizers, such as RMSProp, Adagrad, and Adam. It even supports approximate second-order methods like L-BFGS! You can refer to the [doc](http://pytorch.org/docs/master/optim.html) for the exact specifications of each optimizer.

To use the Module API, follow the steps below:

1. Subclass `nn.Module`. Give your network class an intuitive name like `TwoLayerFC`. 

2. In the constructor `__init__()`, define all the layers you need as class attributes. Layer objects like `nn.Linear` and `nn.Conv2d` are themselves `nn.Module` subclasses and contain learnable parameters, so that you don't have to instantiate the raw tensors yourself. `nn.Module` will track these internal parameters for you. Refer to the [doc](http://pytorch.org/docs/master/nn.html) to learn more about the dozens of builtin layers. **Warning**: don't forget to call the `super().__init__()` first!

3. In the `forward()` method, define the *connectivity* of your network. You should use the attributes defined in `__init__` as function calls that take tensor as input and output the "transformed" tensor. Do *not* create any new layers with learnable parameters in `forward()`! All of them must be declared upfront in `__init__`. 

After you define your Module subclass, you can instantiate it as an object and call it just like the NN forward function in part II.

ベアボーンPyTorchでは、すべてのパラメータテンソルを手作業で追跡する必要があります。これは少数のテンソルを持つ小さなネットワークでは良いのですが、大きなネットワークで数十から数百のテンソルを追跡するのは非常に不便でエラーが発生しやすくなります。

PyTorchは `nn.Module` APIを提供し、学習可能なパラメータをすべて追跡しながら、任意のネットワークアーキテクチャを定義することができます。後編では、SGDを実装しました。PyTorch は `torch.optim` パッケージも提供しており、 RMSProp, Adagrad, Adam などの一般的なオプティマイザを実装しています。L-BFGSのような近似2次手法もサポートしています！各オプティマイザーの正確な仕様については、[doc](http://pytorch.org/docs/master/optim.html)を参照してください。

モジュールAPIを使用するには、以下の手順に従ってください：

1. `nn.Module` をサブクラス化する。ネットワーククラスに `TwoLayerFC` のような直感的な名前を付ける。

2. コンストラクタ `__init__()` で、必要なレイヤーをクラス属性として定義する。`nn.Linear`や`nn.Conv2d`のようなレイヤーオブジェクトはそれ自体が`nn.Module`のサブクラスであり、学習可能なパラメータを含んでいる。`nn.Module`はこれらの内部パラメータを追跡してくれる。何十ものビルトインレイヤーの詳細については[doc](http://pytorch.org/docs/master/nn.html)を参照してください。**警告**: 最初に `super().__init__()` を呼び出すことを忘れないでください！

3. `forward()`メソッドで、ネットワークの*コネクティビティ*を定義する。`__init__`で定義された属性を、テンソルを入力とし、「変換された 」テンソルを出力する関数呼び出しとして使用します。学習可能なパラメータを持つ新しいレイヤーは `forward()` の中で **作らないで**ください！それらはすべて `__init__` の中で前もって宣言しておく必要がある。

モジュールのサブクラスを定義したら、それをオブジェクトとしてインスタンス化し、パート II の NN forward 関数と同じように呼び出すことができる。

### Module API: Two-Layer Network
Here is a concrete example of a 2-layer fully connected network:

ここでは、2層の完全接続ネットワークの具体例を示す：

In [15]:
class TwoLayerFC(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        # レイヤーオブジェクトをクラス属性に割り当てる
        self.fc1 = nn.Linear(input_size, hidden_size)
        # nn.init パッケージには、便利な初期化メソッドが含まれています。
        # http://pytorch.org/docs/master/nn.html#torch-nn-init
        nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        nn.init.kaiming_normal_(self.fc2.weight)

    def forward(self, x):
        # フォワードは常にコネクティビティを定義する
        x = flatten(x)
        scores = self.fc2(F.relu(self.fc1(x)))
        return scores


def test_TwoLayerFC():
    input_size = 50
    x = torch.zeros((64, input_size), dtype=dtype)  # ミニバッチサイズ64、特徴次元50
    model = TwoLayerFC(input_size, 42, 10)
    scores = model(x)
    print(scores.size())  # [64,10]


test_TwoLayerFC()

torch.Size([64, 10])


### Module API: Three-Layer ConvNet
It's your turn to implement a 3-layer ConvNet followed by a fully connected layer. The network architecture should be the same as in Part II:

1. Convolutional layer with `channel_1` 5x5 filters with zero-padding of 2
2. ReLU
3. Convolutional layer with `channel_2` 3x3 filters with zero-padding of 1
4. ReLU
5. Fully-connected layer to `num_classes` classes

You should initialize the weight matrices of the model using the Kaiming normal initialization method.

次はあなたが3層のConvNetを実装し、その後に完全接続層を実装する番だ。ネットワーク・アーキテクチャはパートIIと同じでなければならない：

1. チャンネル_1`の5x5フィルタを持つ畳み込み層。
2. ReLU
3. チャンネル_2`の3x3フィルターによる畳み込み層。
4. ReLU
5. num_classes`クラスへの完全連結層

Kaiming 正規初期化法を用いてモデルの重み行列を初期化する必要がある。

**HINT**: http://pytorch.org/docs/stable/nn.html#conv2d

After you implement the three-layer ConvNet, the `test_ThreeLayerConvNet` function will run your implementation; it should print `(64, 10)` for the shape of the output scores.

3層ConvNetを実装したら、`test_ThreeLayerConvNet`関数で実装を実行する。

In [16]:
class ThreeLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()
        ########################################################################
        # TODO: Set up the layers you need for a three-layer ConvNet with the  #
        # architecture defined above.                                          #
        #                                                                      #
        # 以前に定義したアーキテクチャを持つ3層ConvNetに必要なレイヤーを設定します。
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.conv1 = nn.Conv2d(in_channel, channel_1, 5, padding=2)
        nn.init.kaiming_normal_(self.conv1.weight)
        self.conv2 = nn.Conv2d(channel_1, channel_2, 3, padding=1)
        nn.init.kaiming_normal_(self.conv2.weight)
        self.fc = nn.Linear(channel_2 * 32 * 32, num_classes)
        nn.init.kaiming_normal_(self.fc.weight)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                          END OF YOUR CODE                            #
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: Implement the forward function for a 3-layer ConvNet. you      #
        # should use the layers you defined in __init__ and specify the        #
        # connectivity of those layers in forward()                            #
        #                                                                      #
        # 3 層 ConvNet の forward 関数を実装する。
        # init__で定義したレイヤーを使い、forward()でレイヤーの接続性を指定する。
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        scores = self.fc(flatten(x))

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################
        return scores


def test_ThreeLayerConvNet():
    x = torch.zeros(
        (64, 3, 32, 32), dtype=dtype
    )  # ミニバッチサイズ64、画像サイズ[3, 32, 32］
    model = ThreeLayerConvNet(in_channel=3, channel_1=12, channel_2=8, num_classes=10)
    scores = model(x)
    print(scores.size())  # [64, 10]


test_ThreeLayerConvNet()

torch.Size([64, 10])


### Module API: Check Accuracy
Given the validation or test set, we can check the classification accuracy of a neural network. 

This version is slightly different from the one in part II. You don't manually pass in the parameters anymore.

検証セットまたはテストセットが与えられれば、ニューラルネットワークの分類精度をチェックすることができる。

このバージョンはパート II のものとは少し異なります。手動でパラメータを渡すことはもうありません。

In [17]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on validation set")
    else:
        print("Checking accuracy on test set")
    num_correct = 0
    num_samples = 0
    model.eval()  # モデルを評価モードに設定する
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # GPUなどのデバイスに移す
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print("Got %d / %d correct (%.2f)" % (num_correct, num_samples, 100 * acc))

### Module API: Training Loop
We also use a slightly different training loop. Rather than updating the values of the weights ourselves, we use an Optimizer object from the `torch.optim` package, which abstract the notion of an optimization algorithm and provides implementations of most of the algorithms commonly used to optimize neural networks.

また、学習ループも少し変えている。これは最適化アルゴリズムの概念を抽象化し、ニューラルネットワークの最適化によく使われるアルゴリズムのほとんどを実装したものです。

In [18]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.

    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for

    Returns: Nothing, but prints model accuracies during training.

    PyTorchモジュールAPIを使ってCIFAR-10のモデルを学習します。

    入力
    - モデル： 学習するモデルを指定する PyTorch モジュール。
    - optimizer: モデルの学習に使用する Optimizer オブジェクト。
    - epochs: (オプション) 学習するエポック数を指定する Python 整数。

    戻り値： 何も返さないが、学習中のモデルの精度を表示する。
    """
    model = model.to(device=device)  # モデルのパラメーターをCPU/GPUに移す
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # モデルをトレーニングモードにする
            x = x.to(device=device, dtype=dtype)  # GPUなどのデバイスに移す
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # オプティマイザが更新する変数の勾配をすべてゼロにする。
            optimizer.zero_grad()

            # モデルの各パラメーターに対する損失の勾配を計算する。
            loss.backward()

            # バックワードパスで計算された勾配を使用して、モデルのパラメータを実際に更新する。
            optimizer.step()

            if t % print_every == 0:
                print("Iteration %d, loss = %.4f" % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

### Module API: Train a Two-Layer Network
Now we are ready to run the training loop. In contrast to part II, we don't explicitly allocate parameter tensors anymore.

Simply pass the input size, hidden layer size, and number of classes (i.e. output size) to the constructor of `TwoLayerFC`. 

You also need to define an optimizer that tracks all the learnable parameters inside `TwoLayerFC`.

You don't need to tune any hyperparameters, but you should see model accuracies above 40% after training for one epoch.

これでトレーニングループを実行する準備ができた。パート II とは対照的に、もう明示的にパラメータ・テンソルを確保する必要はない。

単に `TwoLayerFC` のコンストラクタに入力サイズ、隠れ層サイズ、クラス数（つまり出力サイズ）を渡すだけでよい。

また、`TwoLayerFC` の内部で学習可能な全てのパラメータを追跡するオプティマイザを定義する必要がある。

ハイパーパラメータをチューニングする必要はないが、1エポックの学習でモデルの精度が40%以上になるはずだ。

In [19]:
hidden_layer_size = 4000
learning_rate = 1e-2
model = TwoLayerFC(3 * 32 * 32, hidden_layer_size, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_part34(model, optimizer)

Iteration 0, loss = 3.0544
Checking accuracy on validation set
Got 169 / 1000 correct (16.90)

Iteration 100, loss = 1.8644
Checking accuracy on validation set
Got 314 / 1000 correct (31.40)

Iteration 200, loss = 2.0979
Checking accuracy on validation set
Got 350 / 1000 correct (35.00)

Iteration 300, loss = 2.0779
Checking accuracy on validation set
Got 382 / 1000 correct (38.20)

Iteration 400, loss = 2.0383
Checking accuracy on validation set
Got 380 / 1000 correct (38.00)

Iteration 500, loss = 1.7252
Checking accuracy on validation set
Got 451 / 1000 correct (45.10)

Iteration 600, loss = 1.7181
Checking accuracy on validation set
Got 425 / 1000 correct (42.50)

Iteration 700, loss = 1.8181
Checking accuracy on validation set
Got 432 / 1000 correct (43.20)



### Module API: Train a Three-Layer ConvNet
You should now use the Module API to train a three-layer ConvNet on CIFAR. This should look very similar to training the two-layer network! You don't need to tune any hyperparameters, but you should achieve above above 45% after training for one epoch.

You should train the model using stochastic gradient descent without momentum.

モジュールAPIを使用して、CIFAR上で3層のConvNetをトレーニングします。これは2層ネットワークの学習とよく似ているはずだ！ハイパーパラメータを調整する必要はありませんが、1エポックのトレーニングで45%以上を達成する必要があります。

モメンタムなしの確率的勾配降下を使ってモデルをトレーニングする必要があります。

In [20]:
learning_rate = 3e-3
channel_1 = 32
channel_2 = 16

model = None
optimizer = None
################################################################################
# TODO: Instantiate your ThreeLayerConvNet model and a corresponding optimizer #
#                                                                              #
# ThreeLayerConvNetモデルと対応するオプティマイザをインスタンス化する。
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model = ThreeLayerConvNet(3, channel_1, channel_2, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

train_part34(model, optimizer)

Iteration 0, loss = 3.2799
Checking accuracy on validation set
Got 104 / 1000 correct (10.40)

Iteration 100, loss = 2.0869
Checking accuracy on validation set
Got 359 / 1000 correct (35.90)

Iteration 200, loss = 1.7398
Checking accuracy on validation set
Got 397 / 1000 correct (39.70)

Iteration 300, loss = 1.7231
Checking accuracy on validation set
Got 423 / 1000 correct (42.30)

Iteration 400, loss = 1.5849
Checking accuracy on validation set
Got 465 / 1000 correct (46.50)

Iteration 500, loss = 1.5273
Checking accuracy on validation set
Got 454 / 1000 correct (45.40)

Iteration 600, loss = 1.6065
Checking accuracy on validation set
Got 463 / 1000 correct (46.30)

Iteration 700, loss = 1.5060
Checking accuracy on validation set
Got 498 / 1000 correct (49.80)



# Part IV. PyTorch Sequential API

Part III introduced the PyTorch Module API, which allows you to define arbitrary learnable layers and their connectivity. 

For simple models like a stack of feed forward layers, you still need to go through 3 steps: subclass `nn.Module`, assign layers to class attributes in `__init__`, and call each layer one by one in `forward()`. Is there a more convenient way? 

Fortunately, PyTorch provides a container Module called `nn.Sequential`, which merges the above steps into one. It is not as flexible as `nn.Module`, because you cannot specify more complex topology than a feed-forward stack, but it's good enough for many use cases.

パートIIIではPyTorchのモジュールAPIを紹介し、任意の学習可能なレイヤーとその接続性を定義できるようにしました。

フィードフォワード層のスタックのような単純なモデルの場合、3つのステップを踏む必要があります: `nn.Module` をサブクラス化し、`__init__` でレイヤーをクラス属性に割り当て、`forward()` で各レイヤーを1つずつ呼び出します。もっと便利な方法はないでしょうか？

幸いなことに、PyTorchには `nn.Sequential` というコンテナモジュールがあり、上記のステップを1つにまとめることができる。フィードフォワードスタックよりも複雑なトポロジーを指定できないので、`nn.Module`ほど柔軟ではありませんが、多くのユースケースでは十分です。

### Sequential API: Two-Layer Network
Let's see how to rewrite our two-layer fully connected network example with `nn.Sequential`, and train it using the training loop defined above.

Again, you don't need to tune any hyperparameters here, but you shoud achieve above 40% accuracy after one epoch of training.

2層完全連結ネットワークの例を`nn.Sequential`で書き換えて、上で定義した学習ループを使って学習する方法を見てみよう。

ここでもハイパーパラメータを調整する必要はないが、1エポックの学習で40%以上の精度を達成する必要がある。

In [21]:
# nn.Sequentialでスタックするには、`flatten`関数をモジュールでラップする必要がある。
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)


hidden_layer_size = 4000
learning_rate = 1e-2

model = nn.Sequential(
    Flatten(),
    nn.Linear(3 * 32 * 32, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, 10),
)

# optim.SGDではネステロフ・モメンタム(Nesterov momentum)を使うことができる。
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

train_part34(model, optimizer)

Iteration 0, loss = 2.3242
Checking accuracy on validation set
Got 164 / 1000 correct (16.40)

Iteration 100, loss = 2.1967
Checking accuracy on validation set
Got 362 / 1000 correct (36.20)

Iteration 200, loss = 2.0128
Checking accuracy on validation set
Got 405 / 1000 correct (40.50)

Iteration 300, loss = 1.7144
Checking accuracy on validation set
Got 431 / 1000 correct (43.10)

Iteration 400, loss = 2.0279
Checking accuracy on validation set
Got 440 / 1000 correct (44.00)

Iteration 500, loss = 2.0311
Checking accuracy on validation set
Got 430 / 1000 correct (43.00)

Iteration 600, loss = 1.8658
Checking accuracy on validation set
Got 437 / 1000 correct (43.70)

Iteration 700, loss = 1.9248
Checking accuracy on validation set
Got 439 / 1000 correct (43.90)



### Sequential API: Three-Layer ConvNet
Here you should use `nn.Sequential` to define and train a three-layer ConvNet with the same architecture we used in Part III:

1. Convolutional layer (with bias) with 32 5x5 filters, with zero-padding of 2
2. ReLU
3. Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
4. ReLU
5. Fully-connected layer (with bias) to compute scores for 10 classes

You can use the default PyTorch weight initialization.

You should optimize your model using stochastic gradient descent with Nesterov momentum 0.9.

Again, you don't need to tune any hyperparameters but you should see accuracy above 55% after one epoch of training.

ここでは、`nn.Sequential`を使用して、パートIIIで使用したのと同じアーキテクチャで3層のConvNetを定義し、訓練する必要があります：

1. 畳み込み層（バイアス付き）、32個の5x5フィルター、ゼロパディングは2
2. ReLU
3. 16個の3x3フィルターによる畳み込み層（バイアスあり）。
4. ReLU
5. 10クラスのスコアを計算するための完全連結層（バイアスあり

デフォルトのPyTorchの重みの初期化を使うことができます。

Nesterov momentum 0.9で確率勾配降下を使ってモデルを最適化する必要があります。

繰り返しますが、ハイパーパラメータを調整する必要はありませんが、1エポックの学習で55%以上の精度が得られるはずです。

In [22]:
channel_1 = 32
channel_2 = 16
learning_rate = 1e-2

model = None
optimizer = None

################################################################################
# TODO: Rewrite the 2-layer ConvNet with bias from Part III with the           #
# Sequential API.                                                              #
#                                                                              #
# Part IIIのバイアスを持つ2層ConvNetをSequential APIで書き直してください。
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 5, padding=2),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2 * 32 * 32, 10),
)

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

train_part34(model, optimizer)

Iteration 0, loss = 2.3228
Checking accuracy on validation set
Got 145 / 1000 correct (14.50)

Iteration 100, loss = 1.6335
Checking accuracy on validation set
Got 394 / 1000 correct (39.40)

Iteration 200, loss = 1.4801
Checking accuracy on validation set
Got 435 / 1000 correct (43.50)

Iteration 300, loss = 1.3882
Checking accuracy on validation set
Got 507 / 1000 correct (50.70)

Iteration 400, loss = 1.2891
Checking accuracy on validation set
Got 529 / 1000 correct (52.90)

Iteration 500, loss = 1.4207
Checking accuracy on validation set
Got 543 / 1000 correct (54.30)

Iteration 600, loss = 1.0361
Checking accuracy on validation set
Got 559 / 1000 correct (55.90)

Iteration 700, loss = 1.1862
Checking accuracy on validation set
Got 546 / 1000 correct (54.60)



# Part V. CIFAR-10 open-ended challenge

In this section, you can experiment with whatever ConvNet architecture you'd like on CIFAR-10. 

Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves **at least 70%** accuracy on the CIFAR-10 **validation** set within 10 epochs. You can use the check_accuracy and train functions from above. You can use either `nn.Module` or `nn.Sequential` API. 

Describe what you did at the end of this notebook.

Here are the official API documentation for each component. One note: what we call in the class "spatial batch norm" is called "BatchNorm2D" in PyTorch.

* Layers in torch.nn package: http://pytorch.org/docs/stable/nn.html
* Activations: http://pytorch.org/docs/stable/nn.html#non-linear-activations
* Loss functions: http://pytorch.org/docs/stable/nn.html#loss-functions
* Optimizers: http://pytorch.org/docs/stable/optim.html

このセクションでは、CIFAR-10 で好きな ConvNet アーキテクチャを試すことができます。

あとは、アーキテクチャ、ハイパーパラメータ、損失関数、オプティマイザを試して、CIFAR-10 の **検証** セットで 10 エポック以内に **少なくとも 70%** の精度を達成するモデルを学習するのがあなたの仕事です。上記の check_accuracy と train 関数を使用することができます。API は `nn.Module` または `nn.Sequential` のいずれかを使用する。

このノートの最後に何を行ったかを記述する。

各コンポーネントの公式APIドキュメントを以下に示す。注意: クラス 「spatial batch norm」 で呼んでいるものは PyTorch では 「BatchNorm2D」 と呼ばれています。

* torch.nnパッケージのレイヤー: http://pytorch.org/docs/stable/nn.html
* 活性化: http://pytorch.org/docs/stable/nn.html#non-linear-activations
* 損失関数: http://pytorch.org/docs/stable/nn.html#loss-functions
* 最適化： http://pytorch.org/docs/stable/optim.html


### Things you might try:
- **Filter size**: Above we used 5x5; would smaller filters be more efficient?
- **Number of filters**: Above we used 32 filters. Do more or fewer do better?
- **Pooling vs Strided Convolution**: Do you use max pooling or just stride convolutions?
- **Batch normalization**: Try adding spatial batch normalization after convolution layers and vanilla batch normalization after affine layers. Do your networks train faster?
- **Network architecture**: The network above has two layers of trainable parameters. Can you do better with a deep network? Good architectures to try include:
    - [conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [conv-relu-conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [batchnorm-relu-conv]xN -> [affine]xM -> [softmax or SVM]
- **Global Average Pooling**: Instead of flattening and then having multiple affine layers, perform convolutions until your image gets small (7x7 or so) and then perform an average pooling operation to get to a 1x1 image picture (1, 1 , Filter#), which is then reshaped into a (Filter#) vector. This is used in [Google's Inception Network](https://arxiv.org/abs/1512.00567) (See Table 1 for their architecture).
- **Regularization**: Add l2 weight regularization, or perhaps use Dropout.

- フィルターのサイズ 上記では5x5を使いましたが、もっと小さいフィルターの方が効率的でしょうか？
- フィルターの数**： 上記では32個のフィルターを使用した。もっと多い方がいいのか、少ない方がいいのか？
- プーリング対ストライドコンボリューション**： 最大プーリングを使うか、ストライドコンボリューションだけを使うか？
- バッチ正規化**： コンボリューション層の後に空間バッチ正規化、アフィン層の後にバニラバッチ正規化を追加してみてください。ネットワークはより速く学習しますか？
- ネットワーク・アーキテクチャ**： 上記のネットワークは、学習可能なパラメータを2層持っています。ディープ・ネットワークを使えばもっとうまくいくでしょうか？試すのに良いアーキテクチャは以下の通り：
    - [conv-relu-pool]xN→[affine]xM→[softmaxまたはSVM]。
    - [conv-relu-conv-relu-pool]xN→[affine]xM→[softmaxまたはSVM]。
    - [バッチノルム-relu-conv]xN -> [アフィン]xM -> [softmax または SVM]
- グローバル平均プーリング**： 平坦化してから複数のアフィン層を持つ代わりに、画像が小さくなるまで（7x7くらい）畳み込みを行い、平均プーリング演算を行って1x1の画像（1, 1 , Filter#）にし、それを（Filter#）ベクトルに再形成する。これは[Google's Inception Network](https://arxiv.org/abs/1512.00567)で使用されている（アーキテクチャは表1を参照）。
- 正則化**： l2重み正則化を加えるか、あるいはDropoutを使う。

### Tips for training
For each network architecture that you try, you should tune the learning rate and other hyperparameters. When doing this there are a couple important things to keep in mind:

- If the parameters are working well, you should see improvement within a few hundred iterations
- Remember the coarse-to-fine approach for hyperparameter tuning: start by testing a large range of hyperparameters for just a few training iterations to find the combinations of parameters that are working at all.
- Once you have found some sets of parameters that seem to work, search more finely around these parameters. You may need to train for more epochs.
- You should use the validation set for hyperparameter search, and save your test set for evaluating your architecture on the best parameters as selected by the validation set.

試行するネットワーク・アーキテクチャごとに、学習率とその他のハイパーパラメータを調整する必要がある。この際、2つほど重要なことがある：

- パラメータがうまく機能していれば、数百回の反復で改善が見られるはずである。
- ハイパーパラメータチューニングの粗いものから細かいものへのアプローチを覚えておくこと：まず、ほんの数回のトレーニング反復で、広範囲のハイパーパラメータをテストし、まったく機能しないパラメータの組み合わせを見つけることから始める。
- うまくいきそうなパラメータ・セットがいくつか見つかったら、これらのパラメータの周辺をより細かく探索する。より多くのエポック数を訓練する必要があるかもしれない。
- 検証セットはハイパーパラメータの探索に使用し、テストセットは検証セットで選択された最適なパラメータでアーキテクチャを評価するために保存しておくべきである。

### Going above and beyond
If you are feeling adventurous there are many other features you can implement to try and improve your performance. You are **not required** to implement any of these, but don't miss the fun if you have time!

- Alternative optimizers: you can try Adam, Adagrad, RMSprop, etc.
- Alternative activation functions such as leaky ReLU, parametric ReLU, ELU, or MaxOut.
- Model ensembles
- Data augmentation
- New Architectures
  - [ResNets](https://arxiv.org/abs/1512.03385) where the input from the previous layer is added to the output.
  - [DenseNets](https://arxiv.org/abs/1608.06993) where inputs into previous layers are concatenated together.
  - [This blog has an in-depth overview](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)

もしあなたが冒険心が旺盛なら、パフォーマンスを向上させるために、他にも多くの機能を実装することができます。どれも実装する必要はありませんが、時間があれば楽しんでください！

- 代替オプティマイザ： Adam、Adagrad、RMSpropなどを試すことができます。
- リーキーReLU、パラメトリックReLU、ELU、MaxOutなどの代替活性化関数。
- モデル・アンサンブル
- データの増強
- 新しいアーキテクチャ
  - ResNets](https://arxiv.org/abs/1512.03385)では、前の層の入力が出力に追加される。
  - DenseNets](https://arxiv.org/abs/1608.06993)では、前の層の入力が連結される。
  - このブログに詳細な概要がある](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)


### Have fun and happy training! 

In [23]:
# 論文 https://arxiv.org/abs/1608.06993 に基づくDenseNetアプローチ
# Implementation details: https://amaarora.github.io/2020/08/02/densenets.html

from collections import OrderedDict


class _Transition(nn.Sequential):
    """各密なブロックの間に使用されるトランジション層"""

    def __init__(self, in_channels, out_channels):
        """
        Initializes the transition layer.

        Only `1` convolutional layer is used with kernel size of `1` to reduce the
        _depth_ of the _activation maps_ from `in_channels` to `out_channels`.

        Args:
            in_channels (int): The number of input channels
            out_channels (int): The number of output channels

        遷移層を初期化する。

        カーネルサイズ `1` で `1` 畳み込み層だけが使われる。
        活性化マップの深さを `in_channels` から `out_channels` にする。

        引数
            in_channels (int): 入力チャンネルの数
            out_channels (int): 出力チャンネル数
        """
        super().__init__()

        # 生の入力をbatchnormとreluに通してアクティブ度を得る
        self.add_module("norm", nn.BatchNorm2d(in_channels))
        self.add_module("relu", nn.ReLU(inplace=True))

        # チャンネルと空間領域のダウンサンプリングを実行する
        self.add_module("conv", nn.Conv2d(in_channels, out_channels, 1, bias=False))
        self.add_module("pool", nn.AvgPool2d(2, 2))


class _DenseLayer(nn.Module):
    """各高密度ブロックに使用されるバルディング層"""

    def __init__(self, in_channels, growth_rate, bottleneck_size, drop_rate):
        """
        Initializes the dense layer.

        The layer takes in a batch of inputs of depth `in_channels`, and,
        instead of producing `growth_rate` feature maps, it firstly reduces
        the input depth to `bottleneck_size * growth_rate` (by performing a
        convolution with filter size `(1, 1)`) and only then performs the main
        convolution with filter size `(3, 3)`.

        Args:
            in_channels (int):     The number of input channels
            growth_rate (int):     The number of output channels
            bottleneck_size (int): The bottleneck before main convolution
            drop_rate (int):       The dropout hyperparameter

        密なレイヤーを初期化する。

        この層は深さ `in_channels` の入力を一度に取り込む、
        この層は `growth_rate` の特徴マップを生成する代わりに、まず入力の深さを
        の特徴マップを生成する代わりに、まず入力の深さを `bottleneck_size * growth_rate` まで減少させる。
        フィルターサイズ `(1, 1)`での畳み込みを行う)。
        コンボリューションを行う。

        引数
            in_channels (int):     入力チャンネル数
            growth_rate (int):     出力チャンネル数
            bottleneck_size (int): メイン畳み込み前のボトルネック
            drop_rate (int):       ドロップアウトのハイパーパラメータ
        """
        super().__init__()

        # 最初の深度ダウンサンプリングを行うボトルネックレイヤー
        self.norm1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(
            in_channels, growth_rate * bottleneck_size, 1, bias=False
        )

        # 特徴を抽出し、成長率活性化マップを作成するメインレイヤー
        self.norm2 = nn.BatchNorm2d(growth_rate * bottleneck_size)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            growth_rate * bottleneck_size, growth_rate, 3, padding=1, bias=False
        )

        # 我々はドロップアウトを使用することがある。
        self.drop_rate = float(drop_rate)

    def forward(self, x):
        """
        Performs forward pass on the given input.

        Args:
            x (Tensor): Input data of dim (M, N, in_channels, H, W)

        Returns:
            output (Tensor): Output data of dim (N, growth_rate, H, W)

        与えられた入力に対してフォワードパスを行う。

        引数
            x (テンソル): dim (M, N, in_channels, H, W)の入力データ。

        戻り値
            output (テンソル): dim (N, growth_rate, H, W)の出力データ。
        """
        x = [x] if torch.is_tensor(x) else x  # テンソルであることを保証する。
        x = self.conv1(
            self.relu1(self.norm1(torch.cat(x, 1)))
        )  # 最初の（ボトルネックとなる）パスを実行する
        output = self.conv2(self.relu2(self.norm2(x)))  # 2回目のパス（特徴抽出）を行う

        # ドロップアウト率が提示されている場合は、ドロップアウトを使用する
        if self.drop_rate > 0:
            output = F.dropout(output, p=self.drop_rate, training=self.training)

        return output


class _DenseBlock(nn.ModuleDict):
    """DenseNetで使用されるビルディング・ブロック"""

    def __init__(
        self, num_layers, in_channels, growth_rate, bottleneck_size, drop_rate
    ):
        """
        Initializes the dense block.

        The block constructs `num_layers` densely connected layers with shared
        hyperparameters, taking into account that the output of each layer is
        fed into _all_ other subsequent layers.

        Args:
            num_layers (int):      The number of layers this block will have
            in_channels (int):     The number of input channels
            growth_rate (int):     The number of output channels for each layer
            bottleneck_size (int): The bottleneck before main convolution for each layer
            drop_rate (float):     The dropout hyperparameter

        密なブロックを初期化する。

        このブロックは `num_layers` 個の密結合レイヤーを共有ハイパーパラメータで構成する。
        ハイパーパラメータを持つ密結合層を構築する。
        各層の出力が後続する他の全ての層に供給されることを考慮する。

        引数
            num_layers (int):      このブロックが持つ層の数。
            in_channels (int):     入力チャンネルの数。
            growth_rate (int):     各レイヤーの出力チャンネル数
            bottleneck_size (int): 各層のメイン畳み込み前のボトルネック。
            drop_rate (float):     ドロップアウトのハイパーパラメータ
        """
        super().__init__()

        # すべてのレイヤーをループし、イニシャル化する。
        for i in range(num_layers):
            layer = _DenseLayer(
                in_channels + i * growth_rate, growth_rate, bottleneck_size, drop_rate
            )
            self.add_module(f"denselayer{i+1}", layer)

    def forward(self, x):
        """
        Performs forward pass for each layer.

        Args:
            x (Tensor): Input data of dim (N, in_channels, H, W)

        Returns:
            Output data of dim (N, growth_rate * num_layers, H, W)

        各レイヤーに対してフォワードパスを行う。

        引数
            x (Tensor): dim (N, in_channels, H, W)の入力データ。

        戻り値
            dim (N, growth_rate * num_layers, H, W) の出力データ。
        """
        xs = [x]  # 入力として連結されるテンソルのリストを使う。

        # 各レイヤーをループし、入力として前の出力を連結する。
        for name, layer in self.items():
            x_new = layer(xs)
            xs.append(x_new)

        return torch.cat(xs, 1)


class DenseNet(nn.Module):
    """
    Densely connected network

    The network has the following architecture:
        1. `CONV->NORM->RELU` to preprocess the input for a chain of dense blocks
        2. `BLOCK->[TRANS->BLOCK] x N` where each block consists of arbitrary
           number of densely connected layers
        3. `NORM->RELU->POOL->LINEAR` where global average pooling is performed
           before calculating raw scores

    高密度に接続されたネットワーク

    このネットワークは以下のようなアーキテクチャを持つ：
        1. CONV->NORM->RELU`で密なブロックの連鎖の入力を前処理する。
        2. BLOCK->[TRANS->BLOCK] x N` ここで、各ブロックは任意の数の密結合層で構成される。
           ここで、各ブロックは任意の数の密結合層で構成される。
        3. NORM->RELU->POOL->LINEAR` ここでグローバル平均プーリングが実行される。
           生のスコアを計算する前に
    """

    def __init__(
        self,
        in_channels=32,
        growth_rate=16,
        bottleneck_size=4,
        block_config=(6, 12, 8),
        drop_rate=0,
        num_classes=10,
    ):
        """
        Initializes the dense network.

        The first layer produces `in_channels` activation maps which are then fed to a
        sequence of dense blocks containing a specified number of layers. There is a
        transition layer between each block. At the end the _global average pooling_
        layer is used to flatten the activations for the linear softmax classifier.

        Args:
            block_config (tuple):  The number of layers each bloack should have in sequence
            in_channels (int):     The number of input channels for the sequence of blocks
            growth_rate (int):     The number of output channels for each layer per block
            bottleneck_size (int): The bottleneck before main convolution for each layer per block
            drop_rate (float):     The dropout hyperparameter
            num_classes (int):     The total number of classes

        密なネットワークを初期化する。

        最初の層は `in_channels` 活性化マップを生成し、それが指定された数の層を含む一連の密なブロックに供給される。
        指定された数の層を含む密なブロックのシーケンスに供給される。各ブロックの間には
        各ブロックの間には遷移層がある。最後に_global average pooling_ 層がある。
        層は線形ソフトマックス分類器のための活性化を平坦化するために使われる。

        引数
            block_config (タプル):  各ブロックが順番に持つべき層の数。
            in_channels (int):     一連のブロックの入力チャンネル数。
            growth_rate (int):     ブロックごとの各層の出力チャンネル数
            bottleneck_size (int): ブロックごとの各レイヤーのメイン畳み込み前のボトルネック。
            drop_rate (float):     ドロップアウトのハイパーパラメータ
            num_classes (int):     クラスの総数
        """
        super().__init__()

        # 入力を前処理するためにレイヤーを初期化する（空間的次元を保持する）
        self.features = nn.Sequential(
            OrderedDict(
                [
                    ("conv0", nn.Conv2d(3, in_channels, 7, padding=3, bias=False)),
                    ("norm0", nn.BatchNorm2d(in_channels)),
                    ("relu0", nn.ReLU(inplace=True)),
                ]
            )
        )

        num_features = in_channels  # 各ブロックで更新される入力サイズ

        # 指定された数のブロックを作成する（32x32のスペーシャルディムの場合、3でなければならない）
        for i, num_layers in enumerate(block_config):
            # 密なブロックを作成、追加し、次のブロックのチャンネル数を更新する。
            block = _DenseBlock(
                num_layers, num_features, growth_rate, bottleneck_size, drop_rate
            )
            self.features.add_module(f"denseblock{i+1}", block)
            num_features += num_layers * growth_rate

            # 最後のブロックでない場合はトランジションを追加する。
            if i != len(block_config) - 1:
                trans = _Transition(num_features, num_features // 2)
                self.features.add_module(f"transition{i+1}", trans)
                num_features = num_features // 2

        # グローバル平均プーリングの前に、最後のバッチノルム層とReLU層を追加する。
        self.features.add_module(f"norm{i+2}", nn.BatchNorm2d(num_features))
        self.features.add_module(f"relu{i+2}", nn.ReLU(inplace=True))

        # 最後の層は線形分類器
        self.classifier = nn.Linear(num_features, num_classes)

        # torch リポジトリからの公式 init
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        """
        Performs forward pass for the whole network.

        Args:
            x (Tensor): input data of dim (N, 3, H, W)

        Returns:
            Output data of dim (N, 10)

        ネットワーク全体のフォワードパスを実行する。

        引数:
            x (Tensor): dim (N, 3, H, W)の入力データ

        戻り値:
            dim (N, 10)の出力データ
        """
        out = F.adaptive_avg_pool2d(
            self.features(x), (1, 1)
        )  # アクティブ度をフラットにするグローバル平均プーリング
        out = self.classifier(flatten(out))  # 各クラスの生得点を生成する分類器

        return out

In [24]:
################################################################################
# TODO:                                                                        #
# Experiment with any architectures, optimizers, and hyperparameters.          #
# Achieve AT LEAST 70% accuracy on the *validation set* within 10 epochs.      #
#                                                                              #
# Note that you can use the check_accuracy function to evaluate on either      #
# the test set or the validation set, by passing either loader_test or         #
# loader_val as the second argument to check_accuracy. You should not touch    #
# the test set until you have finished your architecture and  hyperparameter   #
# tuning, and only run the test set once at the end to report a final value.   #
#                                                                              #
# あらゆるアーキテクチャ、オプティマイザ、ハイパーパラメータで実験する。
# 10エポック以内に検証セットで少なくとも70%の精度を達成する。
#
# check_accuracy関数を使用して、
# テストセットまたは検証セットで評価することができます。
# check_accuracyに2番目の引数としてloader_testまたはloader_valを渡すことで、
# テストセットまたは検証セットで評価することができます。
# アーキテクチャとハイパーパラメータの調整が完了するまで、
# テストセットには触れないでください。
# 最後にテストセットを1回実行して最終値を報告してください。
################################################################################
model = None
optimizer = None

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

learning_rate = 0.0015

model = DenseNet()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

# 少なくとも70％の精度が必要
train_part34(model, optimizer, epochs=10)

Iteration 0, loss = 2.3174
Checking accuracy on validation set
Got 98 / 1000 correct (9.80)

Iteration 100, loss = 1.5924
Checking accuracy on validation set
Got 357 / 1000 correct (35.70)

Iteration 200, loss = 1.4333
Checking accuracy on validation set
Got 384 / 1000 correct (38.40)

Iteration 300, loss = 1.4819
Checking accuracy on validation set
Got 474 / 1000 correct (47.40)

Iteration 400, loss = 1.2201
Checking accuracy on validation set
Got 533 / 1000 correct (53.30)

Iteration 500, loss = 1.3145
Checking accuracy on validation set
Got 512 / 1000 correct (51.20)

Iteration 600, loss = 1.4328
Checking accuracy on validation set
Got 584 / 1000 correct (58.40)

Iteration 700, loss = 0.9993
Checking accuracy on validation set
Got 601 / 1000 correct (60.10)

Iteration 0, loss = 1.2352
Checking accuracy on validation set
Got 589 / 1000 correct (58.90)

Iteration 100, loss = 1.0129
Checking accuracy on validation set
Got 612 / 1000 correct (61.20)

Iteration 200, loss = 0.8462
Checkin

## Describe what you did 

In the cell below you should write an explanation of what you did, any additional features that you implemented, and/or any graphs that you made in the process of training and evaluating your network.

下のセルには、あなたが行ったことの説明、追加で実装した機能、ネットワークのトレーニングと評価の過程で作成したグラフなどを記入してください。

**Answer:**

### Network design
I used a **DenseNet** ([paper](https://arxiv.org/abs/1608.06993)) with the following architecture:
1. `CONV->NORM->RELU`
    * As the input is of shape `32x32x3`, initially I used a convolution layer with filter size `7x7` to produce `32` _activation maps_. Padding of `3` was used to preserve the input spacial dimension.
    * The `32` _activation maps_, each of shape `32x32`, were then passed through _batchnorm_ layer and through _ReLU_ nonlinearity
2. `BLOCK->TRANS->BLOCK->TRANS->BLOCK`
    * In general each block has a certain number of layers, each of which perfom a _convolution_, _batchnorm_ and _ReLU_. Each layer concatinates its input with every other layer's output within the same block. Throughout the convolutions in the block, the spacial size of each _activation map_ is preserved, however, the channel size grows by a constant of `16` after each layer produces an output.
    * Each transition layer simply performs a _convolution_ to reduce the channel size by a factor of `2`. We also have _average pooling_ whith reducs the spatial dimension also by a factor of `2`. As in any other group of layers, there is a _normalization_ layer and _ReLU_ nonlinearity.
    * I used `6` layers in the first block, `12` in the second and `8` in the final one. Thus the 'preprocessed' input of size `32x32x32` after each block and transition had its shape modified in had its dimensions changed in the following way `32x32x128->16x16x64->16x16x256->8x8x128->8x8x256` (based on the provided sequence of blocks and transitions).
3. `NORM->RELU->POOL->SOFTMAX`
    * The raw activation maps are passed through _batchnorm_ and _ReLU_, a _global average pooling_ is performed to get the final activation shape of `1x1x256` which is then flattened to a linear classifier which produces scores based on the _Softmax_ loss function.

> Key motivation is that by allowing subsequent layers to see the outputs of every previous layer it makes the network more rubust in recognizing certain details. For example, if one layer detects edges and a subsequent layer detects shapes, sometimes it may be better to see the shape before deciding on the edge.

私は以下のアーキテクチャを持つ**DenseNet**（[論文](https://arxiv.org/abs/1608.06993)）を使用した：

1. Conv->norm->relu`
    * 入力は `32x32x3` の形をしているので、最初は `7x7` のフィルターサイズの畳み込みレイヤーを使って `32` の _activation maps_ を生成した。入力の空間次元を保持するために、`3`のパディングを使用した。
    * 32x32`の形状の `32` _activation maps_ は、次に_batchnorm_レイヤーを通過し、_ReLU_非線形性を通過しました。
2. ブロック->トランス->ブロック->トランス->ブロック`。
    * 一般的に各ブロックはある数のレイヤーを持ち、各レイヤーは_convolution_、_batchnorm_、_ReLU_を実行する。各レイヤーは、その入力を同じブロック内の他のレイヤーの出力と連結する。ブロック内の畳み込みを通して、それぞれの_activation map_の空間サイズは保持されるが、各レイヤーが出力を生成した後、チャンネルサイズは定数`16`だけ大きくなる。
    * 各移行層は、チャンネル・サイズを`2`倍に縮小するための_convolution_を実行するだけである。また、空間次元を`2`倍に縮小する_平均プーリング_もある。他の層のグループと同様に、正規化層とReLU非線形性があります。
    * 最初のブロックでは`6`層、2番目のブロックでは`12`層、最後のブロックでは`8`層を使用した。このように、各ブロックと遷移の後のサイズ `32x32x32` の'前処理済み'入力は、その形状が変更され、その寸法が次のように変更された `32x32x128->16x16x64->16x16x256->8x8x128->8x8x256` （提供されたブロックと遷移のシーケンスに基づく）。
3. norm->relu->pool->softmax`。
    * 生の活性化マップが_batchnorm_と_ReLU_に渡され、_global average pooling_が実行され、最終的な活性化シェイプである`1x1x256`が得られます。

> 主な動機は、後続のレイヤーが前のレイヤーの出力をすべて見ることができるようにすることで、ネットワークが特定の詳細をより正確に認識できるようになることだ。例えば、ある層がエッジを検出し、後続の層が形状を検出する場合、エッジを決定する前に形状を見た方が良い場合がある。

### Hyperparameters
I only fine-tuned 3 hyperparameters - _learning rate_, _weight decay_ and _drop rate_. As it turns out, dropout and regularization do not impact performance much, in many cases - even make it worse, possibly due to heavy use of batch normalization which already make sthe network robust. I settled on the learning rate of `0.0015`, larger values make the validation accuracy too unstable during training.

私が微調整したのは3つのハイパーパラメーターだけで、_learning rate_、_weight decay_、_drop rate_だ。結局のところ、ドロップアウトと正則化は性能にあまり影響を与えず、多くの場合、むしろ悪化させる。おそらくバッチ正規化を多用することで、すでにネットワークがロバストになっているためだろう。私は学習率を`0.0015`に設定した。これより大きな値を設定すると、学習中の検証精度が不安定になりすぎるからだ。

## Test set -- run this only once

Now that we've gotten a result we're happy with, we test our final model on the test set (which you should store in best_model). Think about how this compares to your validation set accuracy.

満足のいく結果が得られたので、テストセット（best_modelに格納します）で最終モデルをテストします。検証セットの精度と比較してみてください。

In [25]:
best_model = model
check_accuracy_part34(loader_test, best_model)

Checking accuracy on test set
Got 8349 / 10000 correct (83.49)
